In [ ]:
import os
import numpy as np
import keras.layers
import matplotlib.pyplot as plt
#import tensorflow as tf
from sklearn.metrics import mean_squared_error
import h5py
import scipy
import skimage
import random
import csv
import cv2
import index_tracker as track  
from keras.callbacks import LearningRateScheduler
import time

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# set seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)


from MaskRCNN import maskrcnn_model as modellib, data_generator as dg, config as conf, visualize, utils, metrics
from mrcnn import test_mrcnn

**1. Set training hyperparameters**

In [ ]:
# config
config = conf.Config()
config.NAME = 'lung_tumor'
config.BACKBONE = "resnet50"
config.BATCH_SIZE =  8
config.IMAGES_PER_GPU = 8
TRAIN_BN = None
config.IMAGE_HEIGHT = 384
config.IMAGE_WIDTH = 384
config.IMAGE_SHAPE = [384, 384, 1]
config.MASK = False
config.FPN_CLASSIF_FC_LAYERS_SIZE = 256
config.DETECTION_MIN_CONFIDENCE = 0.1

data_path = ''   
model = 'mrcnn'
data_mode = 'standard'

**2. Load training and validation data**

In [ ]:
# load data
patient_name = 'patient1'
model_path = data_path + patient_name + '/models/' + model + '/' + data_mode
drr_data_path = model_path + '/dataset/'    
LOG_DIR = model_path + '/logs/'

imgs = load_data(drr_data_path + 'train_imgs.hdf5', 'train_imgs')
labs = load_data(drr_data_path + 'train_labs.hdf5', 'train_labs')
test_imgs = load_data(drr_data_path + 'test_imgs.hdf5', 'test_imgs')
test_labs = load_data(drr_data_path + 'test_labs.hdf5', 'test_labs')

**3. Preprocess loaded data**

In [ ]:
# get the center 384x384 region
imgs = imgs[..., 64:-64]
labs = labs[..., 64:-64]
test_imgs = test_imgs[..., 64:-64]
test_labs = test_labs[..., 64:-64]

for i in range(labs.shape[0]):
    for j in range(labs.shape[1]):
        labs[i, j] = fit_ellipse(labs[i, j])
for i in range(test_labs.shape[0]):
    for j in range(test_labs.shape[1]):
        test_labs[i, j] = fit_ellipse(test_labs[i, j]) 
imgs = (imgs+1)/2.
test_imgs = (test_imgs+1)/2.

# TRAIN MODEL
s = test_imgs.shape
imgs = np.reshape(imgs, (-1, s[2], s[3]))
labs = np.reshape(labs, (-1, s[2], s[3]))
test_imgs = np.reshape(test_imgs, (-1, s[2], s[3]))
test_labs = np.reshape(test_labs, (-1, s[2], s[3]))

**4. Define training model**

In [ ]:
# load model
model = modellib.MaskRCNN(mode="training", model_dir=LOG_DIR,
                          config=config)    

**5. Train model**

In [ ]:
def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch<=10 and lr < 1e-2:
        lr = lr + lr/epoch
    elif epoch > 10 and lr > 1e-5:
        k = 0.05
        lr = lr*np.exp(-k*epoch)
    return lr
lrs = LearningRateScheduler(lr_scheduler)  

model.train(imgs, labs, test_imgs, test_labs,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='all',
                custom_callbacks = [lrs])